In [7]:
import os
import pathlib
import logging
import argparse
from minio import Minio
from minio.error import ResponseError
from kubernetes import client
from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1ResourceRequirements

class ApplyModelKfserving(object):
    def copy_model_version(self, namespace, version):

        # by using default minio
        mc = Minio("minio-service.kubeflow.svc.cluster.local:9000",
                    access_key="minio",
                    secret_key="minio123",
                    secure=False)  
        model_bucket_name = f'{namespace}-model-result'
        print("model_bucket_name " + model_bucket_name) 
        bucket_exists = mc.bucket_exists(model_bucket_name)
        if bucket_exists:
            lists = mc.list_objects(model_bucket_name, 
                                    prefix=f"result/saved_model/{version}", 
                                    recursive=True)
            copy_model_dir = "/model/result/saved_model"
            pathlib.Path(copy_model_dir).mkdir(parents=True, exist_ok=True)
            for object in lists:
                if os.path.isdir(copy_model_dir):
                    mc.fget_object(model_bucket_name, object.object_name, "/model/" + object.object_name)    
                    
            for (path, dir, files) in os.walk(copy_model_dir):
                for filename in files:
                    print("%s/%s" % (path, filename))
        else:
            raise Exception("bucket not exists")
            
    
    
    def apply_kfserving(self, model_name, namespace, pvc_name):
        api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
        storage_uri = f'pvc://{pvc_name}/result/saved_model'
        print("Stroage_uri " + storage_uri)
        default_endpoint_spec = V1alpha2EndpointSpec(
                                  predictor=V1alpha2PredictorSpec(
                                    tensorflow=V1alpha2TensorflowSpec(
                                      storage_uri=storage_uri,
                                      resources=V1ResourceRequirements(
                                          requests={'cpu':'100m','memory':'1Gi'},
                                          limits={'cpu':'100m', 'memory':'1Gi'}))))

        isvc = V1alpha2InferenceService(api_version=api_version,
                                  kind=constants.KFSERVING_KIND,
                                  metadata=client.V1ObjectMeta(
                                      name=model_name, namespace="kubeflow"),
                                  spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))        
        KFServing = KFServingClient()
        # if model server is exist
        if KFServing.get(model_name, namespace="kubeflow"):
            KFServing.patch(model_name, isvc)
            print("Patched KFServing service")
        else:
            KFServing.create(isvc)
            print("Created KFServing service")
            
        KFServing.get(model_name, namespace="kubeflow", watch=True, timeout_seconds=360) 
        
    def apply(self):
        parser = argparse.ArgumentParser()
        parser.add_argument('--model_name', required=False, type=str, default="mnist_kfserving")
        parser.add_argument('--namespace', required=False, type=str, default="amaramusic")        
        parser.add_argument('--pvc_name', required=False, type=str, default="model_pvc")        
        parser.add_argument('--version', required=False, type=str, default="001")
        args = parser.parse_args()     
        
        print(f"{args.pvc_name}")
        
        self.copy_model_version(args.namespace, args.version)
        self.apply_kfserving(args.model_name, args.namespace, args.pvc_name)
        

if __name__=="__main__":
    if os.getenv('FAIRING_RUNTIME', None) is None:
        """
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils

        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'
        fairing.config.set_builder(
            'append',
            image_name='mnist-katib-job',
            base_image='brightfly/katib-sdk:0.0.1',
            registry=DOCKER_REGISTRY,
            push=True)
        # cpu 1, memory 1GiB
        fairing.config.set_deployer('job',
                                    namespace='amaramusic'
                                    )
        fairing.config.run()        
        """
        from kubeflow.fairing.builders.append.append import AppendBuilder
        from kubeflow.fairing.preprocessors.converted_notebook import ConvertNotebookPreprocessor

        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'
        base_image='brightfly/kubeflow-jupyter-lab:tf2.0-cpu'
        image_name='apply-model-kfserving'

        builder = AppendBuilder(
            registry=DOCKER_REGISTRY,
            image_name=image_name,
            base_image=base_image,
            push=True,
            
            preprocessor=ConvertNotebookPreprocessor(
                notebook_file="apply_model_kfserving.ipynb" )
            )
        builder.build()         
    else:
        serving = ApplyModelKfserving()
        serving.apply()        
        
        

[W 201011 14:25:14 append:50] Building image using Append builder...
[I 201011 14:25:14 base:105] Creating docker context: /tmp/fairing_context_4am3xv9i
[I 201011 14:25:15 converted_notebook:127] Converting apply_model_kfserving.ipynb to apply_model_kfserving.py
[I 201011 14:25:15 docker_creds_:234] Loading Docker credentials for repository 'brightfly/kubeflow-jupyter-lab:tf2.0-cpu'
[W 201011 14:25:17 append:54] Image successfully built in 2.718331407988444s.
[W 201011 14:25:17 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/apply-model-kfserving:CEF7B942...
[I 201011 14:25:17 docker_creds_:234] Loading Docker credentials for repository 'kubeflow-registry.default.svc.cluster.local:30000/apply-model-kfserving:CEF7B942'
[W 201011 14:25:17 append:81] Uploading kubeflow-registry.default.svc.cluster.local:30000/apply-model-kfserving:CEF7B942
[I 201011 14:25:17 docker_session_:280] Layer sha256:d846ead9aa38bc0d35dd500ada5349fe42efb96c89db08b8abcb9ad949332c2a exists